In [12]:
#https://github.com/ava-orange-education/Mastering-Computer-Vision-with-PyTorch-2.0 

In [13]:
import torch
import torch.nn as nn

In [14]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()
        
        self.conv_1 = nn.Conv2d(in_channels = 3,
                               out_channels = 32,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)
        
        self.activation = nn.ReLU()
        
        #class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        self.pool = nn.MaxPool2d(kernel_size = 2,stride = 2)
        
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(32*16*16,10) #32*16*16 = 8192, 10
    
    def forward(self, x):
        x = self.conv_1(x)
        x = self.activation(x)
        x = self.pool(x)
        
        x = self.flatten(x)
        
        x = self.fc_1(x)
        
        return x
        
model = SimpleCNN()

In [15]:
# Training Neural Networks

# OPTIM: Contains optimization algorithms like SGD, Adam, and RMSprop.
# You use this to update your model's weights based on the calculated gradients.
import torch.optim as optim

# FUNCTIONAL: Contains "stateless" functions that don't hold weights.
# Commonly used for activation functions (F.relu), loss functions (F.cross_entropy), 
# and pooling operations that don't need to store internal parameters.
import torch.nn.functional as F

In [ ]:
'''
 =========================================================================
                     KINGS OF LOSS FUNCTIONS (CRITERIA)
 =========================================================================

 --- REGRESSION (Predicting continuous numbers) ---

 1. nn.MSELoss() 
 Use: Default for regression. Penalizes large errors heavily (squares them).
 Case: House prices, stock values, temperature.

 2. nn.L1Loss()
 Use: Robust to outliers. Doesn't freak out as much as MSE over "crazy" data.
 Case: When your dataset is "noisy" or has extreme outlier values.

 3. nn.HuberLoss()
 Use: The hybrid. Acts like MSE for small errors and L1 for large ones.
 Case: Best for general regression where you want stability.


 --- CLASSIFICATION (Predicting categories/labels) ---

 ⭐ 4. nn.CrossEntropyLoss()
 Use: The Multi-Class King. Combines LogSoftmax and NLLLoss in one step.
 Case: Image classification (0-9 digits, Cat vs Dog vs Bird). 
 Note: Do NOT add a Softmax layer at the end of your model if you use this.

 ⭐ 5. nn.BCEWithLogitsLoss()
 Use: Binary Cross Entropy. For 2-choice problems.
 Case: Yes/No, Spam/Not Spam, or predicting multiple labels for one image.

 6. nn.NLLLoss() (Negative Log Likelihood)
 Use: Only if you manually put 'F.log_softmax' at the end of your model.
 Case: Advanced classification where you need the log-probabilities.


 --- ADVANCED / SPECIALIZED ---

 7. nn.KLDivLoss() (Kullback-Leibler Divergence)
 Use: Measures how one probability distribution differs from another.
 Case: Knowledge Distillation or Variational Autoencoders (VAEs).

 8. nn.TripletMarginLoss()
 Use: Similarity learning. Makes "friends" closer and "enemies" farther apart.
 Case: Face recognition (FaceID), Signature verification.

 9. nn.CTCLoss() (Connectionist Temporal Classification)
 Use: Predicting sequences when lengths don't match.
 Case: Speech-to-text, Handwriting recognition. 
'''

In [16]:
# Define a loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
'''
 =========================================================================
                       KINGS OF OPTIMIZERS (THE DRIVERS)
 =========================================================================

 1. optim.SGD (Stochastic Gradient Descent)
 Use: The "Old School" classic. Reliable but can be slow.
 Case: Often used with 'momentum=0.9' to help it roll over small bumps in the loss.
 Bro-Tip: Many top-tier researchers still use this for final "polishing" of models.

 2. optim.Adam (Adaptive Moment Estimation)
 Use: The "Default" King. Usually the best starting point for any project.
 Case: Handles different learning rates for different parameters automatically.
 Bro-Tip: Use this if you want results fast without tweaking much.

 3. optim.AdamW (Adam with Weight Decay)
 Use: The "Modern" Adam. It fixes a math bug in how Adam handles weight decay.
 Case: Standard for Transformers and modern CNNs.
 Bro-Tip: If Adam is overfitting, switch to AdamW.

 4. optim.RMSprop (Root Mean Square Propagation)
 Use: Great for "unstable" gradients.
 Case: Frequently used in Recurrent Neural Networks (RNNs) and Reinforcement Learning.

 5. optim.Adagrad (Adaptive Gradient Algorithm)
 Use: Good for sparse data (where some features don't appear often).
 Case: Natural Language Processing (NLP) or recommendation systems.
'''

In [17]:
#Define an optimizer
optimizer = optim.SGD(model.parameters(),lr = 0.001, momentum = 0.9)
#class torch.optim.SGD(params, lr=0.001, momentum=0, dampening=0, weight_decay=0, nesterov=False, *, maximize=False, foreach=None, differentiable=False, fused=None)

In [18]:
#Dummy Datasset

# inputs: Simulates a batch of 64 color images.
# Shape: [64, 3, 32, 32] 
# -> 64: Batch size (how many images we show the AI at once)
# -> 3:   Channels (Red, Green, Blue)
# -> 32, 32: Height and Width in pixels
inputs = torch.randn(64, 3, 32, 32) 

# labels: Simulates the "correct answers" for those 100 images.
# torch.randint(0, 10, (64,))
# -> 0: The lowest possible category index
# -> 10: The highest category index (exclusive, so 0-9)
# -> (64,): A 1D list of 64 random integers (one for each input image)
labels = torch.randint(0, 10, (64,))

In [19]:
# Training Loop

for epoch in range(10):
    running_loss = 0.0
    for i in range(64):
        # Placeholder to show where data enters; usually used to move data to GPU/CPU.
        inputs, labels = inputs, labels #these data are randomly generated in our case 
        
        optimizer.zero_grad() # Clears previous gradients so they don't accumulate.
        
        outputs = model(inputs) # Forward pass: feeds inputs to the model to get predictions.
        loss = criterion(outputs, labels) # Calculates the error between predictions and actual labels.
        
        loss.backward() # Computes the gradient (how to change weights) using backpropagation.
        optimizer.step() # Updates the model parameters based on the computed gradients.
        
        running_loss = running_loss + loss.item() # Adds the current batch's loss to the total.
        
        if i % 10 == 9:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0 # Resets the counter to track the loss for the next 10 iterations.

[1, 10] loss: 2.047
[1, 20] loss: 1.175
[1, 30] loss: 0.474
[1, 40] loss: 0.199
[1, 50] loss: 0.109
[1, 60] loss: 0.075
[2, 10] loss: 0.054
[2, 20] loss: 0.046
[2, 30] loss: 0.041
[2, 40] loss: 0.036
[2, 50] loss: 0.033
[2, 60] loss: 0.030
[3, 10] loss: 0.027
[3, 20] loss: 0.025
[3, 30] loss: 0.023
[3, 40] loss: 0.022
[3, 50] loss: 0.021
[3, 60] loss: 0.020
[4, 10] loss: 0.018
[4, 20] loss: 0.017
[4, 30] loss: 0.016
[4, 40] loss: 0.016
[4, 50] loss: 0.015
[4, 60] loss: 0.014
[5, 10] loss: 0.014
[5, 20] loss: 0.013
[5, 30] loss: 0.013
[5, 40] loss: 0.012
[5, 50] loss: 0.012
[5, 60] loss: 0.011
[6, 10] loss: 0.011
[6, 20] loss: 0.011
[6, 30] loss: 0.010
[6, 40] loss: 0.010
[6, 50] loss: 0.010
[6, 60] loss: 0.009
[7, 10] loss: 0.009
[7, 20] loss: 0.009
[7, 30] loss: 0.009
[7, 40] loss: 0.008
[7, 50] loss: 0.008
[7, 60] loss: 0.008
[8, 10] loss: 0.008
[8, 20] loss: 0.008
[8, 30] loss: 0.007
[8, 40] loss: 0.007
[8, 50] loss: 0.007
[8, 60] loss: 0.007
[9, 10] loss: 0.007
[9, 20] loss: 0.007


In [20]:
# Generate Random Test Data
# We'll simulate 500 test images (5 batches of 100)
test_inputs = torch.randn(50, 3, 32, 32)
test_labels = torch.randint(0, 10, (50,))

In [21]:
# Evaluation Setup
correct = 0
total = 0
batch_size = 16

# We use torch.no_grad() because we aren't training (saves memory/compute)
with torch.no_grad():
    # We iterate through our dummy test data in chunks/batches
    for i in range(0, 50, batch_size):
        # Slicing the data to simulate a loader
        images = test_inputs[i : i + batch_size]
        labels = test_labels[i : i + batch_size]
        
        # Forward pass
        # Note: using 'model' to match your training variable name
        outputs = model(images) 
        
        # Get the index of the highest score (the prediction)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 3. Final Output
accuracy = 100 * correct / total
print(f"Accuracy of the network on random test images is: {accuracy:.2f}%")

Accuracy of the network on random test images is: 8.00%


In [ ]:
# Since this test was done on random data model accuracy is low